In [3]:
import requests
from decouple import config
from fastapi import HTTPException
from app.models.Crypto_models.DataModel_crypto import data_asks_bids, Crypto, Data_crypto

CRYPTO_URL = config("URL_CRYPTO", default="https://api.kraken.com/0/public/Depth")

def get_crypto_pair_data1(symbol: str):
    try:
        url = f"{CRYPTO_URL}"
        params = {"pair": symbol}
        headers = {'accept': 'application/json'}
        payload = {}
        request_timeout = 10
        response = requests.get(url, headers=headers, params=params,
                                data=payload, timeout=request_timeout)
        response.raise_for_status()
        data = response.json()

        print(f"statut de la reponse : {response.status_code}")
        print(f"reponse brute : {response.text}")

        if data.get("error") and len(data["error"]) > 0:
            return f"erreur dans data: {data['error']}"
        if data.get("result"):

            couple_crypto = list(data.get("result").keys())[0]
            data_asks = data.get("result").get(couple_crypto).get("asks")
            data_bids = data.get("result").get(couple_crypto).get("bids")

            ### utilsation de liste en comprehension pour innitialiser les dictionnaire qui vont contenir les données asks et bids
            nb_colonnes = len(data_asks[0])
            dictionnaire_asks = {str(i): [] for i in range(nb_colonnes)}
            dictionnaire_bids = {str(i): [] for i in range(nb_colonnes)}
            for liste_data in data_asks:
                for index, valeur in enumerate(liste_data):
                    dictionnaire_asks[str(index)].append(valeur)
                    dictionnaire_bids[str(index)].append(valeur)


            price_asks= dictionnaire_asks["0"]
            volume_asks= dictionnaire_asks["1"]
            temps_asks= dictionnaire_asks["2"]
            price_bids= dictionnaire_bids["0"]
            volume_bids= dictionnaire_bids["1"]
            temps_bids= dictionnaire_bids["2"]



            instance_data_ask = Data_crypto(
                price = price_asks,
                volume = volume_asks,
                temps = temps_asks
            )
            instance_data_bid = Data_crypto(
                price= price_bids,
                volume= volume_bids,
                temps= temps_bids
            )
            instance_data_asks_bids = data_asks_bids(
                asks= instance_data_ask,
                bids= instance_data_bid
            )
            instance_crypto = Crypto(
                couple_crypto= couple_crypto,
                data_dict= instance_data_asks_bids
            )
            return instance_crypto.dict()

        raise HTTPException(
            status_code=500,
            detail="erreur dans le format attendu dans la reponse"
        )

    except requests.exceptions.RequestException as e:
        return f"probleme à l'appel de l'api de kraken : {str(e)}"


In [5]:
aa = get_crypto_pair_data1("XBTUSD")
aa

statut de la reponse : 200
reponse brute : {"error":[],"result":{"XXBTZUSD":{"asks":[["95953.00000","0.306",1732751899],["95953.40000","2.606",1732751890],["95954.00000","0.001",1732751898],["95954.60000","2.606",1732751890],["95955.50000","5.211",1732751890],["95955.70000","2.606",1732751890],["95958.00000","0.012",1732751895],["95959.80000","5.211",1732751890],["95962.60000","0.405",1732751890],["95965.40000","5.211",1732751890],["95968.60000","0.398",1732751895],["95970.70000","0.141",1732751895],["95972.20000","0.405",1732751881],["95972.60000","5.210",1732751890],["95972.80000","0.099",1732751891],["95973.00000","0.006",1732751891],["95973.80000","0.022",1732751891],["95981.30000","0.042",1732751887],["95982.60000","0.245",1732751890],["95983.80000","0.405",1732751826],["95985.30000","0.004",1732751899],["95985.40000","5.210",1732751843],["95986.30000","0.040",1732751850],["95986.70000","0.144",1732751893],["95987.10000","0.261",1732751895],["95988.20000","0.099",1732751897],["959

{'couple_crypto': 'XXBTZUSD',
 'data_dict': {'asks': {'price': [95953.0,
    95953.4,
    95954.0,
    95954.6,
    95955.5,
    95955.7,
    95958.0,
    95959.8,
    95962.6,
    95965.4,
    95968.6,
    95970.7,
    95972.2,
    95972.6,
    95972.8,
    95973.0,
    95973.8,
    95981.3,
    95982.6,
    95983.8,
    95985.3,
    95985.4,
    95986.3,
    95986.7,
    95987.1,
    95988.2,
    95989.4,
    95991.3,
    95991.4,
    95995.7,
    95997.1,
    95997.2,
    95999.9,
    96000.0,
    96000.9,
    96002.1,
    96002.9,
    96003.0,
    96003.6,
    96005.3,
    96005.8,
    96009.4,
    96010.8,
    96011.4,
    96012.2,
    96012.7,
    96014.3,
    96018.8,
    96020.8,
    96021.5,
    96024.0,
    96025.6,
    96025.7,
    96033.0,
    96034.9,
    96035.8,
    96036.0,
    96036.8,
    96038.5,
    96040.7,
    96042.4,
    96045.2,
    96046.4,
    96049.2,
    96051.7,
    96055.7,
    96055.9,
    96056.0,
    96056.1,
    96056.5,
    96061.9,
    96062.0,
    

corection de bug api open metheo


In [ ]:
import requests
from decouple import config
import requests_cache
from retry_requests import retry
import openmeteo_requests
import pandas as pd


WEATHER_URL = config("URl_WEATHER", default = "https://api.open-meteo.com/v1/forecast")
url = f"{WEATHER_URL}"

cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)

openmeteo = openmeteo_requests.Client(session = retry_session)
def get_weather_data(latitude: float, longitude: float, opmeteo = openmeteo):
    hourly_params = ["temperature_2m", "rain"]
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": ",".join(hourly_params)
    }
    try:
        responses = opmeteo.weather_api(url, params =params)
    except requests.exceptions.RequestException as e:
        return {"erreur": str(e)}

    data_object = responses[0]
    hourly_object = data_object.Hourly()
    data = {}

    data["time"] = pd.date_range(
        start= pd.to_datetime(hourly_object.Time(), unit = "s", utc = True),
        end= pd.to_datetime(hourly_object.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds= hourly_object.Interval()), inclusive = "left")
    for i in range(hourly_object.VariablesLength()):
        data[hourly_params[i]] = hourly_object.Variables(i).ValuesAsNumpy()

    return data

In [1]:
import requests
from decouple import config
import requests_cache
from retry_requests import retry
import openmeteo_requests
from app.models.weather_models.Datamodel import Hourly, Data
import pandas as pd

# Configuration de l'URL de l'API météo
WEATHER_URL = config('URL_WEATHER', default="https://api.open-meteo.com/v1/forecast")

# Initialisation de la session de cache et de la session de retry
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)

# Initialisation du client Open-Meteo
openmeteo = openmeteo_requests.Client(session=retry_session)

def get_weather_data1(latitude: float, longitude: float, openmeteo=openmeteo) -> Data:
    # Paramètres pour la requête API
    hourly_params = ["temperature_2m", "rain"]
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": ",".join(hourly_params)
    }

    try:
        # Effectuer la requête API
        responses = openmeteo.weather_api(WEATHER_URL, params=params)
    except requests.exceptions.RequestException as e:
        # Gérer les exceptions de requête
        return {"erreur": str(e)}

    # Extraire les données de la réponse
    data_object = responses[0]
    hourly_object = data_object.Hourly()

    # Initialiser le dictionnaire de données
    data = {}

    # Générer la plage de temps
    time = pd.date_range(
        start=pd.to_datetime(hourly_object.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly_object.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly_object.Interval()),
        inclusive="left"
    ).astype(str).to_list()

    # Remplir le dictionnaire de données
    for i in range(hourly_object.VariablesLength()):
        data[hourly_params[i]] = hourly_object.Variables(i).ValuesAsNumpy().tolist()

    # Créer l'objet Hourly
    hourly = Hourly(
        time=time,
        temperature_2m=data["temperature_2m"],
        rain=data["rain"]
    )
    instance_Data= Data(
        donnee=hourly
    )

    # Retourner l'objet Data
    return instance_Data.dict()

In [3]:
data = get_weather_data1(48.8534, 2.3488)
data

{'donnee': {'time': ['2024-11-28 00:00:00+00:00',
   '2024-11-28 01:00:00+00:00',
   '2024-11-28 02:00:00+00:00',
   '2024-11-28 03:00:00+00:00',
   '2024-11-28 04:00:00+00:00',
   '2024-11-28 05:00:00+00:00',
   '2024-11-28 06:00:00+00:00',
   '2024-11-28 07:00:00+00:00',
   '2024-11-28 08:00:00+00:00',
   '2024-11-28 09:00:00+00:00',
   '2024-11-28 10:00:00+00:00',
   '2024-11-28 11:00:00+00:00',
   '2024-11-28 12:00:00+00:00',
   '2024-11-28 13:00:00+00:00',
   '2024-11-28 14:00:00+00:00',
   '2024-11-28 15:00:00+00:00',
   '2024-11-28 16:00:00+00:00',
   '2024-11-28 17:00:00+00:00',
   '2024-11-28 18:00:00+00:00',
   '2024-11-28 19:00:00+00:00',
   '2024-11-28 20:00:00+00:00',
   '2024-11-28 21:00:00+00:00',
   '2024-11-28 22:00:00+00:00',
   '2024-11-28 23:00:00+00:00',
   '2024-11-29 00:00:00+00:00',
   '2024-11-29 01:00:00+00:00',
   '2024-11-29 02:00:00+00:00',
   '2024-11-29 03:00:00+00:00',
   '2024-11-29 04:00:00+00:00',
   '2024-11-29 05:00:00+00:00',
   '2024-11-29 06:00:0

In [29]:
import requests
from decouple import config
import requests_cache
from retry_requests import retry
import openmeteo_requests
import pandas as pd
import numpy as np
'''
Définition de la fonction get_weather_data pour les endpoints par heurs, les hourly data
'''
# Configuration de l'URL de l'API météo
WEATHER_URL = config('URL_WEATHER', default="https://api.open-meteo.com/v1/forecast")

# Initialisation de la session de cache et de la session de retry
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)

# Initialisation du client Open-Meteo
openmeteo = openmeteo_requests.Client(session=retry_session)


def get_weather_data_daily1(latitude: float, longitude, openmeteo=openmeteo):

    '''Température maximale à 2 mètres (°C)
    Température minimale à 2 mètres (°C)
    Durée de la lumière du jour (heures)
    Durée d'ensoleillement direct (heures)
    Indice UV maximal (sans unité)
    Somme des précipitations (mm)
    Quantité de pluie uniquement (mm)
    Quantité de neige (cm)
    Probabilité maximale de précipitations (%)
    Vitesse maximale du vent à 10 mètres (km/h)
    Rafales de vent maximales à 10 mètres (km/h)  '''


    daily_params= ["temperature_2m_max", "temperature_2m_min", "sunrise", "sunset",
                "daylight_duration", "sunshine_duration", "uv_index_max", "precipitation_sum",
                "rain_sum", "precipitation_hours", "wind_speed_10m_max", "wind_gusts_10m_max"]


    params = {
        "latitude": latitude,
        "longitude": longitude,
        "daily": ",".join(daily_params)
    }

    try:
        ''' Récuperation de l'objet retourné par l'api d'openméteo, et iteration sur l'objet
            pour interagir avec les données jounalière récupérées '''

        reponses= openmeteo.weather_api(WEATHER_URL, params=params)
        data_object= reponses[0]
        daily_object= data_object.Daily()



        ## générer une plade de temps au format iso pour récuperer la donnée time

        time = pd.date_range(
            start= pd.to_datetime(daily_object.Time(), unit= "s", utc=True),
            end= pd.to_datetime(daily_object.TimeEnd(), unit= "s", utc=True),
            freq= pd.Timedelta(seconds= daily_object.Interval()),
            inclusive= "left").astype(str).to_list()

        # Initialisation d'un dictionnaire en comprehension pour récuperer chaque donnnée et de les formaté en liste
        data= {
            f"{daily_params[i]}": daily_object.Variables(i).ValuesAsNumpy()
            for i in range(daily_object.VariablesLength())
        }

        return data




    except requests.exceptions.RequestException as e:
        print(f"l'exception lévé est: {e}")


In [41]:
e = get_weather_data_daily1(48.8534, 2.100)
e["sunrise"]

0

In [44]:
import requests
from decouple import config
import requests_cache
from retry_requests import retry
import openmeteo_requests
from app.models.weather_models.Datamodel import Hourly, Data, Datadaily, Data_daily
import pandas as pd
def get_weather_data_daily2(latitude: float, longitude: float, openmeteo=openmeteo) -> Data_daily:

    '''Température maximale à 2 mètres (°C)
    Température minimale à 2 mètres (°C)
    Durée de la lumière du jour (heures)
    Durée d'ensoleillement direct (heures)
    Indice UV maximal (sans unité)
    Somme des précipitations (mm)
    Quantité de pluie uniquement (mm)
    Quantité de neige (cm)
    Probabilité maximale de précipitations (%)
    Vitesse maximale du vent à 10 mètres (km/h)
    Rafales de vent maximales à 10 mètres (km/h)  '''

    daily_params = ["temperature_2m_max", "temperature_2m_min", "sunrise", "sunset",
                    "daylight_duration", "sunshine_duration", "uv_index_max", "precipitation_sum",
                    "rain_sum", "precipitation_hours", "wind_speed_10m_max", "wind_gusts_10m_max"]
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "daily": ",".join(daily_params)
    }

    try:
        ''' Récuperation de l'objet retourné par l'api d'openméteo, et iteration sur l'objet
            pour interagir avec les données jounalière récupérées '''

        reponses= openmeteo.weather_api(WEATHER_URL, params=params)
        data_object= reponses[0]
        daily_object= data_object.Daily()



        ## générer une plage de temps au format iso pour récuperer la donnée time

        time = pd.date_range(
            start= pd.to_datetime(daily_object.Time(), unit= "s", utc=True),
            end= pd.to_datetime(daily_object.TimeEnd(), unit= "s", utc=True),
            freq= pd.Timedelta(seconds= daily_object.Interval()),
            inclusive= "left").astype(str).to_list()

        # Initialisation d'un dictionnaire en comprehension pour récuperer chaque donnnée et de les formaté en liste
        data= {
            f"{daily_params[i]}": daily_object.Variables(i).ValuesAsNumpy()
            for i in range(daily_object.VariablesLength())
        }

        # Création d'intance de classes Datadaily et data pour la validation de schema et un format accepté par API d'openmetéo

        intance_data_daily= Datadaily(
            time= time,
            temperature_2m_max= data["temperature_2m_max"].tolist(),
            temperature_2m_min= data["temperature_2m_min"].tolist(),
            daylight_duration= data["daylight_duration"].tolist(),
            sunshine_duration= data["sunshine_duration"].tolist(),
            uv_index_max= data["uv_index_max"].tolist(),
            precipitation_sum= data["precipitation_sum"].tolist(),
            rain_sum= data["rain_sum"].tolist(),
            precipitation_hours= data["precipitation_hours"].tolist(),
            wind_speed_10m_max= data["wind_speed_10m_max"].tolist(),
            wind_gusts_10m_max= data["wind_gusts_10m_max"].tolist()
        )

        instance_data= Data_daily(
            donnee= intance_data_daily
        )

        return instance_data.dict()

    except requests.exceptions.RequestException as e:
        print(f"l'exception lévé est: {e}")

In [45]:
a = get_weather_data_daily2(48.8534, 2.100)

ValidationError: 1 validation error for Data_daily
données
  Field required [type=missing, input_value={'donnee': Datadaily(time...5, 28.799999237060547])}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing